In [1]:
#On dispose d'une image d'entrée carrée en niveau de gris
from PIL import Image
import numpy as np

def image_to_gray_matrix(image_path):
    """
    Convertit une image en niveaux de gris en une matrice de niveaux de gris.

    :param image_path: Chemin de l'image à convertir.
    :return: Matrice numpy représentant les niveaux de gris de l'image.
    """
    # Charger l'image
    image = Image.open(image_path)

    # Convertir l'image en niveaux de gris si ce n'est pas déjà le cas
    if image.mode != 'L':
        image = image.convert('L')

    # Convertir l'image en matrice numpy
    image_matrix = np.array(image)

    return image_matrix

# Exemple d'utilisation
image_path = 'chemin/vers/votre/image.jpg'
gray_matrix = image_to_gray_matrix(image_path)

print(gray_matrix)






#On répète jusqu'a ce que la distortion moyenne devienne inferieure a un certain seuil
#A la fin de l'étape précédente on obtient ainsi deux vecteurs qui sont les centres de gravité des deux classes. On split ces deux vecteurs selon la ligne 5 encore, on applique l'étape 3 à nouveau sur ces 2 vecteurs. on s'arrete quand le nombre voulu de vecteurs prototypes (la par exemple on en a 4).
#Une fois le dictionnaire formé, on remplace chaque bloc de l'image par son protoype le plus proche au sens de la distance euclidienne.

[[174 174 174 ... 123 123 123]
 [174 174 174 ... 123 123 123]
 [174 174 174 ... 123 123 123]
 ...
 [ 47  47  47 ...  27  27  27]
 [ 47  47  47 ...  27  27  27]
 [ 47  47  47 ...  27  27  27]]


In [ ]:
#On sépare cette image en bloc de taille NxN
def split_matrix_into_blocks(matrix, block_size):
    """
    Sépare une matrice en blocs de taille NxN.

    :param matrix: Matrice numpy à séparer.
    :param block_size: Taille des blocs NxN.
    :return: Liste de blocs de taille NxN.
    """
    # Obtenir les dimensions de la matrice
    rows, cols = matrix.shape

    # Vérifier que la matrice peut être divisée en blocs de taille NxN
    if rows % block_size != 0 or cols % block_size != 0:
        raise ValueError("La taille de la matrice doit être un multiple de la taille des blocs.")

    # Initialiser la liste de blocs
    blocks = []

    # Séparer la matrice en blocs
    for i in range(0, rows, block_size):
        for j in range(0, cols, block_size):
            block = matrix[i:i + block_size, j:j + block_size]
            blocks.append(block)

    return blocks


In [ ]:
#Chaque bloc est mappé en vecteur de dimension N²
def block_to_vector(block):
    """
    Transforme un block de taille N en un vecteur de dimension n² en empilant les colonnes.
    
    """
    # Vérifier que le block est carré
    rows, cols = block.shape
    if rows != cols:
        raise ValueError("La matrice doit être carrée.")

    # Empiler les colonnes de la matrice l'une en dessous de l'autre
    vector = matrix.T.flatten()

    return vector

def matrix_to_vectors(matrix,block_size):
    matrix_blocks = []
    blocks_list = split_matrix_into_blocks()
    for k in range(len(blocks_list)):
        vector = block_to_vector(blocks_list[k])
        matrix_blocks.append(vector)
    return matrix_blocks

In [ ]:
#On détermine le vecteur qui est le centre de gravité c0 de tous les vecteurs
def center_of_gravity(vectors):
    """
    Détermine le centre de gravité d'une liste de vecteurs.

    :param vectors: Liste de vecteurs numpy.
    :return: Vecteur numpy représentant le centre de gravité.
    """
    # Vérifier que la liste de vecteurs n'est pas vide
    if len(vectors) == 0:
        raise ValueError("La liste de vecteurs ne peut pas être vide.")
    
    # Convertir la liste de vecteurs en une matrice numpy pour faciliter les opérations
    vector_matrix = np.array(vectors)
    
    # Calculer la somme de tous les vecteurs
    sum_vector = np.sum(vector_matrix, axis=0)
    
    # Calculer le centre de gravité en divisant par le nombre de vecteurs
    center_vector = sum_vector / len(vectors)
    
    return center_vector

In [ ]:
#On split ce vecteur en 2 vecteurs c01=c0+eps et c11=c0-eps
def split_vector(vector,eps):
    return vector+eps, vector-eps

In [ ]:
#On classe tous les autres vecteurs de la base d'apprentissage relativement a ces 2 vecteurs en utilisant le critère de la distance euclienne minimale.
def class_vector(vector, c01, c11):
    if distance_euclienne(vector, class1)<=distance_euclienne(vector, class2):
        return class1
    else:
        return class2
    
def distance_euclienne(v1, v2):
    return np.linalg.norm(v1-v2)


In [ ]:
#On obtient 2 classes de vecteurs. Pour chaque classe, on calcule le centre de gravité. les deux vecteurs obtenus remplacent c01 et c11.
def average_distortion_between_centroids(vectors1, centroid1, vectors2, centroid2):
    """
    Calcule la distortion moyenne entre deux ensembles de vecteurs par rapport à leurs centres de gravité respectifs.

    Args:
        vectors1 (list): Une liste de vecteurs représentés sous forme de listes ou de tableaux numpy pour le premier ensemble.
        centroid1 (array-like): Le centre de gravité du premier ensemble de vecteurs.
        vectors2 (list): Une liste de vecteurs représentés sous forme de listes ou de tableaux numpy pour le deuxième ensemble.
        centroid2 (array-like): Le centre de gravité du deuxième ensemble de vecteurs.

    Returns:
        float: La distortion moyenne entre les deux ensembles de vecteurs par rapport à leurs centres de gravité respectifs.
    """
    # Convertir les listes de vecteurs en tableaux numpy pour un traitement plus efficace
    vectors1 = np.array(vectors1)
    vectors2 = np.array(vectors2)
    
    # Calculer les distances de chaque vecteur par rapport à leur centre de gravité respectif
    distances1 = np.linalg.norm(vectors1 - centroid1, axis=1)
    distances2 = np.linalg.norm(vectors2 - centroid2, axis=1)
    
    # Calculer la distortion moyenne entre les deux ensembles de vecteurs
    average_distortion = np.mean(np.concatenate((distances1, distances2)))
    
    return average_distortion

In [ ]:
def LBG(image_path, N, delta, n): #N: taille de bloc voulue ; delta: distortion max threshold ; n: nombre voulu de vecteurs prototypes.
    matrix = image_to_gray_matrix(image_path)
    vectors = matrix_to_vectors(matrix,N)    
    eps = np.random.uniform(0.0001, 0.0002, np.sqrt(N))
    c0 = center_of_gravity(vectors)
    prototypes = [c0]
    while len(prototypes) < n:
        c01, c11 = split_vector(c0,eps)
        mean_distortion = 1
        while mean_distortion >= delta:
            class0 = []
            class1 = []
            for k in range(len(vectors)):
            if class_vector(vectors[k], c01, c11)==c01:
                class0.append(vectors[k])
            else:
                class1.append(vectors[k])
            center_class0 = center_of_gravity(class0)
            center_class1 = center_of_gravity(class1)
            mean_distortion = average_distortion_between_centroids(class0, center_class0, class1, center_class1)
            c01 = center_class0
            c11 = center_class1